In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [3]:
ALL_EVALUATIONS="tests/evaluation_08_retrievers_000_009_sampled_50_queries_gpt-4-1106-preview_20231227.tsv"

BM25_RUN="tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt"
BM25_E5_RUN="tests/bm25_e5_docs_000_999_50_sampled_queries_run.txt"
BM25_MT5_RUN="tests/200-queries-passagens-by-BM25-mT5_v4_with_query_id_run.txt"
SPLADE_RUN="tests/splade_pt_result_with_clueweb_id_run.txt"
COLBERTX_RUN="tests/colbertx_all_znormalized_validation_scores_000_199_run.txt"
E5_RUN="tests/e5_docs_000_009_50_sampled_queries_score_inv_run.txt"
E5_BASE_RUN="tests/e5_base_docs_000_009_50_sampled_queries_score_inv_run.txt"

In [4]:
BM25_1M="tests/bm25_1M_docs_000_009_sampled_50_queries.tsv"

In [5]:
retrievers_results = []

### Format GPT-4 evaluations to qrels format

In [6]:
eval_df = pd.read_csv(ALL_EVALUATIONS, sep='\t')

In [7]:
eval_df

,query_id,query,passage_id,mT5,splade,colbertx,bm25_e5,bm25,e5,e5_base,bm25_1M,passage,score,reasoning,usage,cost,duration,saved_cost
0,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-82-13187_0,False,False,True,False,False,False,False,False,"Mamíferos do Brasil - Zoologia | Museu Nacional - UFRJ ➤ Início ➤ Exposições ➤ Zoologia (Acervo) ➤ Insetos ➤ Mamíferos do Brasil Mamíferos do Brasil Uma larga coleção de mamíferos da fauna brasileira. Entre os importantes representantes deste acervo de espécimes taxidermizados encontram-se grandes símbolos da diversidade biológica do Brasil. Bugio, Barbado, Guariba Bugio Cachorro-do-mato Caititu Capivara Caxinguelê, Serelepe Cervo-do-Pantanal Gato-do-Mato-pequeno Irara Lobo-Guará Lontra Mão-Pelada Muriqui Onça-Pintada Ouriço-Cacheiro, Cuandu Paca Preguiça-de-Bentinho Preguiça-de-três-dedos Rato-Coró Saruê, Gambá Sauá, Guigó Tamanduá-bandeira Tamanduá-í Tamanduá-mirim, meleta Tatu-Canastra Tatu-de-rabo-mole Tatu-Galinha Tatu-Peludo Veado Veado-mateiro « Voltar | Voltar ao Topo",1,"A passagem lista vários mamíferos da fauna brasileira, mas não descreve características gerais da fauna brasileira, como a pergunta solicita. A passagem está relacionada ao tema da fauna brasileira, mas não responde diretamente à pergunta sobre a maior característica da fauna.","{'prompt_tokens': 765, 'completion_tokens': 75, 'total_tokens': 840}",0.00000,6.693229,0.02745
1,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-59-04599_2,False,False,True,False,False,False,False,False,"Segundo uma lista levantada pelo Comitê Brasileiro de Registros Ornitológicos (CBRO), algumas das principais espécies encontradas aqui no país são: Bem-te-vi, Arara-azul, Tuiuiú, Tucano e o Papagaio. Só no Cerrado pode-se encontrar por volta de 148 espécies de aves bem específicas da região. Algumas das aves que mais representam esta região brasileira são: Seriemas; Pavãozinho-do-pará; Coruja-orelhuda; Arara-canindé; Coruja-da-igreja; Arara-vermelha-grande; Tuiuiú (considerado o símbolo do Pantanal). Quando chegamos à região nordestina a ave brasileira já foi até tema de música, a famosa ave com seu voo incrível chamada “Asa branca”, ave que deu nome à música de mais sucesso na voz do cantor e compositor Luiz Gonzaga. A Amazônia é uma região brasileira que qualquer um se encanta pelas belezas naturais principalmente quando o assunto são espécies de aves. As araras com sua pelagem colorida chamam a atenção de todos e podem ser vistas por todos os lados na região. A segunda ave que pode ser encontrada facilmente na Amazônia é o tucano, com suas características marcantes, um bico longo e cores vibrantes que marcam a região dos olhos. Claro que podemos encontrar essas bela espécie de ave em zoológicos espalhados por todo o país ou em áreas de matas em geral.",1,"A passagem descreve várias espécies de aves encontradas no Brasil e suas características, mas não resume qual é a maior característica da fauna brasileira de forma geral. A pergunta pede uma característica ampla da fauna, enquanto a passagem foca em exemplos específicos de aves em diferentes regiões.","{'prompt_tokens': 860, 'completion_tokens': 82, 'total_tokens': 942}",0.00000,5.163547,0.03072
2,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-64-06285_39,True,True,False,False,False,False,True,False,"A exuberante fauna e flora brasileiras, relatadas por Pero Vaz de Caminha em sua carta à Coroa Portuguesa, está sendo destruída gradativamente pela ação antrópica, por meio do desmatamento e da caça d... Leia redação completa Redação sem título. 02/07/2016 O Brasil tem a maior biodiversidade do mundo. Devido a sua variedade de espécies silvestres, desencadeou uma enorme ambição global ao tráfico de animais desde o período colonial. Atualmente, tal ato a... Leia redação completa Redação sem título. 19/09/2018 O Brasil com seu vasto território é um dos países com a maior biodiversidade de animais silvestres. Entretanto, não é dado

In [8]:
eval_df.drop_duplicates("query_id")[['query_id', 'query']]

,query_id,query
0,1,Qual a maior característica da fauna brasileira?
54,2,Por que os países Guiana e Suriname não são filiados a Conmebol?
116,9,quais países europeus seguem o regime monarquista?
170,11,quais os critérios de definição dos monumentos intitulados maravilhas do mundo moderno?
218,13,Como o Brasil reagiu a epidemia de AIDS no fim do século XX?
263,15,Qual a maior torcida de futebol do Brasil?
297,17,Quando se realizou o plebiscito popular para definir o sistema político do Brasil?
339,20,Por que a legislação de um país é tão importante?
406,21,Qual o nosso atual sistema de governo?
468,22,Qual o sistema de governo da África do Sul?


#### Check scores distribution on the current qrels

In [9]:
np.unique(eval_df['score'].to_numpy(), return_counts=True)

(array([0, 1, 2, 3]), array([1196,  517,  407,  469]))

In [10]:
for group_name, group_df in eval_df.groupby('query_id'):
    print("\nquery_id={}".format(group_name))
    
    print(np.unique(group_df['score'].to_numpy(), return_counts=True))


query_id=1
(array([0, 1, 2, 3]), array([ 6, 30, 11,  7]))

query_id=2
(array([0, 1, 2, 3]), array([57,  2,  1,  2]))

query_id=9
(array([0, 1, 2, 3]), array([36,  4,  9,  5]))

query_id=11
(array([0, 1, 2, 3]), array([29, 10,  7,  2]))

query_id=13
(array([0, 1, 2, 3]), array([21,  9, 11,  4]))

query_id=15
(array([0, 1, 2, 3]), array([ 9, 12,  1, 12]))

query_id=17
(array([0, 1, 2, 3]), array([11,  4,  1, 26]))

query_id=20
(array([0, 1, 2, 3]), array([10, 35, 18,  4]))

query_id=21
(array([0, 1, 2, 3]), array([32, 15,  1, 14]))

query_id=22
(array([0, 1, 3]), array([34,  8,  5]))

query_id=24
(array([0, 1, 2, 3]), array([36, 17,  9,  2]))

query_id=26
(array([0, 1, 2, 3]), array([13,  5,  8, 23]))

query_id=28
(array([0, 1, 2, 3]), array([33,  9, 13,  4]))

query_id=47
(array([0, 1, 2]), array([29, 21, 16]))

query_id=49
(array([0, 1, 2, 3]), array([18, 17, 15,  1]))

query_id=51
(array([0, 1, 2, 3]), array([17, 14,  8,  1]))

query_id=54
(array([0, 1, 2, 3]), array([ 1, 11,  8, 29]

In [11]:
eval_df.groupby('query_id').count()

,query,passage_id,mT5,splade,colbertx,bm25_e5,bm25,e5,e5_base,bm25_1M,passage,score,reasoning,usage,cost,duration,saved_cost
query_id,,,,,,,,,,,,,,,,,
1,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54
2,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62
9,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54
11,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48
13,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45
15,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34
17,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42
20,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67
21,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62


#### Prepare the qrels format for trec_eval tools

In [12]:
eval_df['0'] = '0'

In [13]:
qrels_filename = os.path.join(os.path.dirname(ALL_EVALUATIONS), "{}_qrel.txt".format(os.path.splitext(os.path.basename(ALL_EVALUATIONS))[0]))

In [14]:
qrels_filename

'tests/evaluation_08_retrievers_000_009_sampled_50_queries_gpt-4-1106-preview_20231227_qrel.txt'

In [15]:
eval_df.sort_values('query_id')[['query_id', '0', 'passage_id', 'score']].to_csv(qrels_filename, header=None, index=False, sep=" ")

### Compute BM25 nDCG@10

In [16]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt

In [17]:
retrievers_results.append({'retriever':'BM25', 'nDCG@10': result[0].split('\t')[-1]})

In [18]:
retrievers_results

[{'retriever': 'BM25', 'nDCG@10': '0.4594'}]

### Compute BM25 + E5 nDCG@10

In [19]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {BM25_E5_RUN}

In [20]:
retrievers_results.append({'retriever':'BM25+E5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 + mT5 nDCG@10

In [21]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {BM25_MT5_RUN}

In [22]:
retrievers_results.append({'retriever':'BM25+mT5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute SPLADE nDCG@10

In [23]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {SPLADE_RUN}

In [24]:
retrievers_results.append({'retriever':'SPLADE_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ColBERTX nDCG@10

In [25]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {COLBERTX_RUN}

In [26]:
retrievers_results.append({'retriever':'ColBERT-X_mMARCO_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 nDCG@10

In [27]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {E5_RUN}

In [28]:
retrievers_results.append({'retriever':'E5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 base nDCG@10

In [29]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {E5_BASE_RUN}

In [30]:
retrievers_results.append({'retriever':'E5 base', 'nDCG@10': result[0].split('\t')[-1]})

### Prepare BM25 1M retrieval results in run format

In [31]:
bm25_1M_df = pd.read_csv(BM25_1M, sep="\t")

In [32]:
bm25_1M_df.groupby("query_id").head(10).count()

query_id       500
Q0             500
colbertx_id    500
position       500
score          500
comment        500
passage_id     500
query          500
passage        500
dtype: int64

In [33]:
bm25_1M_df

,query_id,Q0,colbertx_id,position,score,comment,passage_id,query,passage
0,1,Q0,3783987,1,9.434800,Anserini,clueweb22-pt0000-79-16312_218,Qual a maior característica da fauna brasileira?,"Fonte: http://www.gazetadopovo.com.br/esportes/olimpiadas/conteudo.phtml?tit=Cameron-aposta-na-Olimpiada-para-recuperar-economia&id=1279366 Tecnológica Titulo: Atletas brasileiros utilizam uniformes tecnológicos na Olimpíada Em busca de medalhas nos Jogos Olímpicos de Londres, os atletas brasileiros terão a tecnologia como um fator aliado. Os representantes nacionais no atletismo e no vôlei tiveram tênis especiais desenvolvidos pela Asics, enquanto os atletas da Confederação Brasileira de Tênis (CBT) e da Confederação Brasileira de Handebol (CBHb) contarão com uniformes oficiais da marca. Criados pela estilista Tatiana Mancini, os novos uniformes foram feitos com tecidos inteligentes e tecnológicos. Além disso, as roupas desenvolvidas por Mancini apresentam características especiais que remetem aos principais aspectos do Brasil . Queríamos algo que transparecesse brasilidade só de olhar, como sacar logo de cara. A ideia foi dar um diferencial ainda maior para as peças através das estampas e cores. No uniforme da Seleção Brasileira feminina de handebol, utilizamos animais e toda a fauna brasileira com desenhos de araras com estampas de onça sobreposta”, afirmou a estilista."
1,1,Q0,2630333,2,9.375400,Anserini,clueweb22-pt0000-55-12305_1,Qual a maior característica da fauna brasileira?,"A presença marcante da m ata está no litoral brasileiro, em grande parte do Sudeste, do Sul e de alguns estados do Centro-Oeste e Nordeste. Estima-se que, quando os europeus aqui chegaram, em 1500, essa floresta ocupava 15% to território brasileiro, com uma área de 1.306.421 km². Com o passar dos anos e o intenso desmatamento causado pelas ações antrópicas, esse número é drasticamente menor, com pouco mais de 100 mil km² da vegetação original. Sua localização litorânea e de baixas e médias latitudes dá-lhe o caráter de uma floresta úmida, quente e com fauna e flora bastante desenvolvid a s. Como a ocupação brasileira ocorreu, de forma inicial, na parte leste do país, a mata foi o primeiro bioma a ser desmatado, o que explica sua baixa área. Localizada no litoral brasileiro, a Mata Atlântica foi o primeiro bioma desmatado do Brasil. Subdivisões da Mata Atlântica A Mata Atlântica tem algumas características bem semelhantes às da Floresta Amazônica. Essas características são referentes ao porte das árvores e ao desenvolvimento da fauna: ambos os biomas possuem árvores altas que podem chegar a mais de 30 metros de altura, com pequenas vegetações arbustivas que surgem à sombra delas."
2,1,Q0,7499132,3,9.197300,Anserini,clueweb22-pt0001-56-07770_0,Qual a maior característica da fauna brasileira?,"Peixe tambaqui: conheça a espécie e suas características Peixes Peixe tambaqui: conheça a espécie e suas características Seu nome científico é Colossoma macropomum. O peixe Tambaqui é popularmente conhecido como Pacu Vermelho. Este é o segundo maior peixe de escamas do Rio Amazonas, na região formada pelo regime do rio Solimões e pelo derretimento das geleiras dos Andes. Este é um peixe típico de água doce do Brasil. Habitante das matas inundadas, este peixe é encontrado em Mato Grosso, Goiás, São Paulo e no norte do país. São famosos por viver em águas pobres de minerais e, apesar disso, serem muito nutritivos e resistentes a doenças. Os animais compõem a biodiversidade da fauna das águas doces brasileiras, e atuam na cadeia alimentar como predadores e consumidores, o que infere à carne sabor peculiar e muito apreciado. Alimentação São onívoros, assim como algumas espécies terrestres como os marsupiais, eles comem de tudo. Têm hábitos diurnos, quando ficam ativos à procura de alimentos."
3,1,Q0,8210163,4,9.089900,Anserini,clueweb22-pt0001-71-09093_76,Qual a maior característica da fauna brasileira?,"biomas brasileiros (em quilômetros quadrados) 2% 5% 48% 10% 2% FABIO COLOMBI

In [34]:
bm25_1M_run_df = bm25_1M_df.sort_values(["query_id", "score"], ascending=[True, False]).groupby("query_id").head(10)

In [35]:
bm25_1M_run_df.shape

(500, 9)

In [36]:
bm25_1M_run_df['Q0'] = "Q0"
bm25_1M_run_df['comment'] = "gpt-4-1106-preview"
bm25_1M_run_df['index'] = np.tile(list(range(1, 11)), 50)

In [37]:
bm25_1M_run_df.head(12)

,query_id,Q0,colbertx_id,position,score,comment,passage_id,query,passage,index
0,1,Q0,3783987,1,9.4348,gpt-4-1106-preview,clueweb22-pt0000-79-16312_218,Qual a maior característica da fauna brasileira?,"Fonte: http://www.gazetadopovo.com.br/esportes/olimpiadas/conteudo.phtml?tit=Cameron-aposta-na-Olimpiada-para-recuperar-economia&id=1279366 Tecnológica Titulo: Atletas brasileiros utilizam uniformes tecnológicos na Olimpíada Em busca de medalhas nos Jogos Olímpicos de Londres, os atletas brasileiros terão a tecnologia como um fator aliado. Os representantes nacionais no atletismo e no vôlei tiveram tênis especiais desenvolvidos pela Asics, enquanto os atletas da Confederação Brasileira de Tênis (CBT) e da Confederação Brasileira de Handebol (CBHb) contarão com uniformes oficiais da marca. Criados pela estilista Tatiana Mancini, os novos uniformes foram feitos com tecidos inteligentes e tecnológicos. Além disso, as roupas desenvolvidas por Mancini apresentam características especiais que remetem aos principais aspectos do Brasil . Queríamos algo que transparecesse brasilidade só de olhar, como sacar logo de cara. A ideia foi dar um diferencial ainda maior para as peças através das estampas e cores. No uniforme da Seleção Brasileira feminina de handebol, utilizamos animais e toda a fauna brasileira com desenhos de araras com estampas de onça sobreposta”, afirmou a estilista.",1
1,1,Q0,2630333,2,9.3754,gpt-4-1106-preview,clueweb22-pt0000-55-12305_1,Qual a maior característica da fauna brasileira?,"A presença marcante da m ata está no litoral brasileiro, em grande parte do Sudeste, do Sul e de alguns estados do Centro-Oeste e Nordeste. Estima-se que, quando os europeus aqui chegaram, em 1500, essa floresta ocupava 15% to território brasileiro, com uma área de 1.306.421 km². Com o passar dos anos e o intenso desmatamento causado pelas ações antrópicas, esse número é drasticamente menor, com pouco mais de 100 mil km² da vegetação original. Sua localização litorânea e de baixas e médias latitudes dá-lhe o caráter de uma floresta úmida, quente e com fauna e flora bastante desenvolvid a s. Como a ocupação brasileira ocorreu, de forma inicial, na parte leste do país, a mata foi o primeiro bioma a ser desmatado, o que explica sua baixa área. Localizada no litoral brasileiro, a Mata Atlântica foi o primeiro bioma desmatado do Brasil. Subdivisões da Mata Atlântica A Mata Atlântica tem algumas características bem semelhantes às da Floresta Amazônica. Essas características são referentes ao porte das árvores e ao desenvolvimento da fauna: ambos os biomas possuem árvores altas que podem chegar a mais de 30 metros de altura, com pequenas vegetações arbustivas que surgem à sombra delas.",2
2,1,Q0,7499132,3,9.1973,gpt-4-1106-preview,clueweb22-pt0001-56-07770_0,Qual a maior característica da fauna brasileira?,"Peixe tambaqui: conheça a espécie e suas características Peixes Peixe tambaqui: conheça a espécie e suas características Seu nome científico é Colossoma macropomum. O peixe Tambaqui é popularmente conhecido como Pacu Vermelho. Este é o segundo maior peixe de escamas do Rio Amazonas, na região formada pelo regime do rio Solimões e pelo derretimento das geleiras dos Andes. Este é um peixe típico de água doce do Brasil. Habitante das matas inundadas, este peixe é encontrado em Mato Grosso, Goiás, São Paulo e no norte do país. São famosos por viver em águas pobres de minerais e, apesar disso, serem muito nutritivos e resistentes a doenças. Os animais compõem a biodiversidade da fauna das águas doces brasileiras, e atuam na cadeia alimentar como predadores e consumidores, o que infere à carne sabor peculiar e muito apreciado. Alimentação São onívoros, assim como algumas espécies terrestres como os marsupiais, eles comem de tudo. Têm hábitos diurnos, quando ficam ativos à procura de alimentos.",3
3,1,Q0,8210163,4,9.0899,gpt-4-1106-preview,clueweb22-pt0001-71-09093_76,Qual a maior característica da fauna brasileira?,"biomas brasileiros (em quilômetr

In [39]:
bm25_1M_run_df[['query_id', 'Q0', "passage_id", "index", "score", "comment"]].to_csv("tests/bm25_1M_docs_000_009_50_sampled_queries_run.txt", 
                                                                                     header=None, index=False, sep=" ")

#### Compute BM25 1M NDCG@10

In [40]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} "tests/bm25_1M_docs_000_009_50_sampled_queries_run.txt"

In [41]:
retrievers_results.append({'retriever':'BM25 1M', 'nDCG@10': result[0].split('\t')[-1]})

### Check the overall results

In [42]:
pd.DataFrame(retrievers_results)

,retriever,nDCG@10
0,BM25,0.4594
1,BM25+E5,0.6558
2,BM25+mT5,0.7341
3,SPLADE_pt-BR,0.5963
4,ColBERT-X_mMARCO_pt-BR,0.6473
5,E5,0.5744
6,E5 base,0.4120
7,BM25 1M,0.3263
